In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# Configuración del WebDriver (en este caso, Chrome) para modo headless
options = Options()
options.headless = True  # Ejecutar el navegador en segundo plano

# Inicializar el WebDriver con ChromeDriver Manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Función para desplazarse hacia abajo para cargar productos
def scroll_down():
    driver.execute_script("window.scrollBy(0, 1000);")  # Desplazarse 1000 píxeles hacia abajo
    time.sleep(3)  # Tiempo para esperar que los productos carguen

# Lista para almacenar todos los productos
productos_lista = []

# Función para extraer los productos de la página actual
def extraer_productos():
    productos = driver.find_elements(By.CSS_SELECTOR, 'div.vtex-search-result-3-x-galleryItem')
    for producto in productos:
        try:
            # Extraer el nombre del producto
            nombre_producto = producto.find_element(By.CSS_SELECTOR, 'h2.vtex-product-summary-2-x-productNameContainer span').text
            
            # Extraer el enlace del producto
            url_producto = producto.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
            
            # Extraer la URL de la imagen
            url_imagen = producto.find_element(By.CSS_SELECTOR, 'img').get_attribute('src')
            
            # Extraer el precio final
            try:
                precio_final = producto.find_element(By.CSS_SELECTOR, 'span.vtex-product-price-1-x-sellingPriceValue').text
            except:
                precio_final = ''  # Si no se encuentra, dejamos el precio vacío
            
            # Extraer el precio original (tachado)
            try:
                precio_original = producto.find_element(By.CSS_SELECTOR, 'span.vtex-product-price-1-x-listPriceValue').text
            except:
                precio_original = ''  # Si no se encuentra, dejamos el precio vacío

            # Almacenar los detalles del producto en una lista
            productos_lista.append({
                "Nombre del Producto": nombre_producto,
                "URL del Producto": f"https://cl.hm.com{url_producto}",
                "URL de la Imagen": url_imagen,
                "Precio Final": precio_final,
                "Precio Original": precio_original
            })
        except Exception as e:
            print("Error al extraer los datos del producto:", e)

# Empezar desde la página 1
pagina = 1
while True:
    url = f'https://cl.hm.com/infantil/ninos?page={pagina}'
    print(f"Accediendo a la página {pagina}: {url}")
    
    # Abrir la página
    driver.get(url)
    
    # Esperar que los elementos carguen
    time.sleep(5)
    
    # Desplazamiento hacia abajo para cargar más productos
    previous_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        scroll_down()
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == previous_height:
            break
        previous_height = new_height
    
    # Extraer los productos de la página actual
    extraer_productos()
    
    # Comprobar si hay más productos cargados
    productos_cargados = driver.find_elements(By.CSS_SELECTOR, 'div.vtex-search-result-3-x-galleryItem')
    
    # Si no hay productos, significa que hemos llegado al final
    if not productos_cargados:
        print("No hay más productos para cargar. Finalizando.")
        break
    
    # Incrementar la página para cargar la siguiente
    pagina += 1

# Crear un DataFrame de pandas con los productos extraídos
df = pd.DataFrame(productos_lista)

# Guardar los datos en un archivo Excel
excel_path = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\HIJA.xlsx"
df.to_excel(excel_path, index=False)

print(f"Los productos se han guardado exitosamente en: {excel_path}")

# Cerrar el navegador
driver.quit()


Accediendo a la página 1: https://cl.hm.com/infantil/ninos?page=1
Accediendo a la página 2: https://cl.hm.com/infantil/ninos?page=2
Accediendo a la página 3: https://cl.hm.com/infantil/ninos?page=3
Accediendo a la página 4: https://cl.hm.com/infantil/ninos?page=4
Accediendo a la página 5: https://cl.hm.com/infantil/ninos?page=5
Accediendo a la página 6: https://cl.hm.com/infantil/ninos?page=6
Accediendo a la página 7: https://cl.hm.com/infantil/ninos?page=7
Accediendo a la página 8: https://cl.hm.com/infantil/ninos?page=8
Accediendo a la página 9: https://cl.hm.com/infantil/ninos?page=9
Accediendo a la página 10: https://cl.hm.com/infantil/ninos?page=10
Accediendo a la página 11: https://cl.hm.com/infantil/ninos?page=11
No hay más productos para cargar. Finalizando.
Los productos se han guardado exitosamente en: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\HIJA.xlsx
